In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pprint import pprint
from bs4 import BeautifulSoup  
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pprint import pprint
from bs4 import BeautifulSoup  
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import csv
import re
import seaborn as sns
import numpy as np

In [48]:
nbaList = ["Points", "Rebounds", "Assists", "Pts+Rebs+Asts", "3-PT Made", "Pts+Rebs","Pts+Asts","Rebs+Asts","Turnovers", "Fantasy Score"]
playerList = []
lineList = []

def scrapePrizePicks(Sport):
    for Prop in nbaList:
        try:
            PATH = "C:\Program Files (x86)\chromedriver.exe"
            driver = webdriver.Chrome(PATH)
            driver.get("https://app.prizepicks.com/")
            driver.find_element(By.CLASS_NAME, "close").click()
            driver.find_element(By.XPATH,"//div[@class='name'][normalize-space()='" + Sport + "']").click()
            try:
                driver.implicitly_wait(2)
                driver.find_element(By.XPATH,"//div[@class='stat stat-active'][normalize-space()='" + Prop + "']").click()
            except:
                driver.implicitly_wait(2)
                driver.find_element(By.XPATH,"//div[@class='stat '][normalize-space()='" + Prop + "']").click()

            projections = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".projection")))

            project = []

            for projection in projections:

                names = projection.find_element(By.XPATH,'.//div[@class="name"]').get_attribute('innerHTML')
                line= projection.find_element(By.XPATH,'.//div[@class="presale-score"]').get_attribute('innerHTML')
                team = projection.find_element(By.XPATH,'.//div[@class="team-position"]').get_attribute('innerHTML')
                update = re.search("([A-Za-z]+)\s-\s[A-Z-a-z]*", team).group(1)
                position = re.search("[A-Za-z]+\s-\s([A-Z-a-z]*)", team).group(1)
                opposing = projection.find_element(By.XPATH,'.//div[@class="opponent"]').get_attribute('innerHTML')
                oteam = re.search("vs\s([A-Za-z]*)", opposing).group(1)



                players = {'Name': names, "Position":position, "Team":update, "Opp Team":oteam, Prop:line}

                project.append(players)
                playerList.append(names)
                lineList.append(line)

            df = pd.DataFrame(project)
            df[[Prop]] = df[[Prop]].apply(pd.to_numeric) 
            df = df.sort_values(by = Prop, ascending = False)
            df.to_csv(f"{Prop} Prizepicks.csv")
            driver.quit()
        except:
            continue

In [49]:
scrapePrizePicks("NBA")

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3135225775.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3135225775.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3135225775.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3135225775.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3135225775.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh

In [50]:
nbaprop = ["player-points","player-rebounds","player-assists","player-threes","player-turnovers"]
combo = ["pts-%2B-reb-%2B-ast", "pts-%2b-reb", "pts-%2b-ast", "ast-%2b-reb"]
defense = ["steals-%2b-blocks"]

In [51]:
def draftkings():
    for prop in nbaprop:
        url = f"https://sportsbook.draftkings.com/leagues/basketball/nba?category={prop}"

        PATH = "C:\Program Files (x86)\chromedriver.exe"
        driver = webdriver.Chrome(PATH)
        driver.get(url)


        projections = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//table/tbody/tr")))




        project = []

        for projection in projections:
            try:
                name = projection.find_element(By.CLASS_NAME, "sportsbook-row-name").get_attribute('innerHTML')
                over = projection.find_element(By.CLASS_NAME, "sportsbook-outcome-cell__line").get_attribute('innerHTML')
                odds = projection.find_element(By.XPATH, './/span[@class = "sportsbook-odds american default-color"]').get_attribute('innerHTML')
                if odds[0] == "−":
                    new_odds = odds.replace("−", "")
                    oddf = f"-{new_odds}"
                else:
                    new_odds = odds.replace("+", "")
                    oddf = f"+{new_odds}"
                    
                    
                proj = {"Name":name, "DK Line":over, "DK Odds":oddf}


                project.append(proj)
            except:
                continue

        df = pd.DataFrame(project)
        df.to_csv(f"{prop} DK.csv")
        
    for prop in combo:
        url = f"https://sportsbook.draftkings.com/leagues/basketball/nba?category=player-combos&subcategory={prop}"
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        driver = webdriver.Chrome(PATH)
        driver.get(url)


        projections = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//table/tbody/tr")))




        project = []

        for projection in projections:
            try:
                name = projection.find_element(By.CLASS_NAME, "sportsbook-row-name").get_attribute('innerHTML')
                over = projection.find_element(By.CLASS_NAME, "sportsbook-outcome-cell__line").get_attribute('innerHTML')
                odds = projection.find_element(By.XPATH, './/span[@class = "sportsbook-odds american default-color"]').get_attribute('innerHTML')
                if odds[0] == "−":
                    new_odds = odds.replace("−", "")
                    oddf = f"-{new_odds}"
                else:
                    new_odds = odds.replace("+", "")
                    oddf = f"+{new_odds}"

                proj = {"Name":name, "DK Line":over, "DK Odds":oddf}

                project.append(proj)
            except:
                continue

        df = pd.DataFrame(project)
        df.to_csv(f"{prop} DK.csv")

    for prop in defense:
        url = f"https://sportsbook.draftkings.com/leagues/basketball/nba?category=player-blocks/steals&subcategory={prop}"
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        driver = webdriver.Chrome(PATH)
        driver.get(url)


        projections = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//table/tbody/tr")))




        project = []

        for projection in projections:
            try:
                name = projection.find_element(By.CLASS_NAME, "sportsbook-row-name").get_attribute('innerHTML')
                over = projection.find_element(By.CLASS_NAME, "sportsbook-outcome-cell__line").get_attribute('innerHTML')
                odds = projection.find_element(By.XPATH, './/span[@class = "sportsbook-odds american default-color"]').get_attribute('innerHTML')
                if odds[0] == "−":
                    new_odds = odds.replace("−", "")
                    oddf = f"-{new_odds}"
                else:
                    new_odds = odds.replace("+", "")
                    oddf = f"+{new_odds}"

                proj = {"Name":name, "DK Line":over, "DK Odds":oddf}

                project.append(proj)
            except:
                continue

        df = pd.DataFrame(project)
        df.to_csv(f"{prop} DK.csv")

draftkings()

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\4116708489.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\4116708489.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\4116708489.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\4116708489.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\4116708489.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh

In [52]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.pinnacle.com/en/basketball/nba/matchups#period:0")
driver.implicitly_wait(5)
button = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH,
                                                                             '//a[@class="style_btn__Fs5oS"]')))

num = len(button)
print(num)
driver.close()

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\349444480.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\349444480.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


5
5


In [53]:
def scraper():
    all_data = []
    for i in range(num):
        i=i+3
        PATH = "C:\Program Files (x86)\chromedriver.exe"
        driver = webdriver.Chrome(PATH)
        driver.set_window_size(1920,1080)
        driver.get("https://www.pinnacle.com/en/basketball/nba/matchups#period:0")
        driver.implicitly_wait(5)
        driver.find_element(By.XPATH,'//*[@id="root"]/div/div[1]/div[4]/div/div/div[2]/div[3]/div/div/div').click()
        driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/div[4]/div/div/div[2]/div[3]/div/div/div[2]/div/ul/li[2]').click()
        driver.find_element(By.XPATH,f'//*[@id="root"]/div/div[2]/main/div/div[4]/div[2]/div/div[{i}]/div[6]/a').click()
        try:
            driver.find_element(By.XPATH,'//button[@id = "player-props"]').click()
            time.sleep(5)
            src = driver.page_source
            soup = BeautifulSoup(src, "lxml")
            playerdata = soup.find("div", attrs = {"class": "style_marketGroups__2htUW matchup-market-groups"})
            playerboxes = playerdata.find_all("div", attrs={"class": "style_primary__3IwKt style_marketGroup__1-qlF"})
            for playerbox in playerboxes:
                if playerbox.find("svg", attrs={"class": "style_offline__3AUfP"}):
                    playerbox.extract()
            playername = playerdata.findAll("span", attrs = {"class": "style_titleText__35WhH"})
            playerprop = playerdata.findAll("span", attrs = {"class": "style_label__2KJur"})
            playerodds = playerdata.findAll("span", attrs = {"class": "style_price__15SlF"})
            players = [players.text for players in playername]
            props = [props.text for props in playerprop]
            odds = [odds.text for odds in playerodds]
            newprops = []
            for i in range(0, len(props), 2):
                newprops.append(props[i])
            newodds = []
            for i in range(0, len(odds), 2):
                newodds.append(odds[i])
            names = []
            for player in players:
                match = re.search(r'^([^(]+)', player)
                if match:
                    name = match.group(1).strip()
                    names.append(name)
            stats = []
            for player in players:
                match = re.search(r'\(([^)]+)\)', player)
                if match:
                    stats.append(match.group(1))
            values = []
            for prop in newprops:
                match = re.search(r'Over (\d+(?:\.\d+)?)', prop)
                if match:
                    values.append(float(match.group(1)))
            combined_list = list(zip(names, stats, values, newodds))
            columns = ['Name', 'Prop', 'Pin Line', 'Pinnacle Odds']
            df = pd.DataFrame(combined_list, columns=columns)
            df['Pinnacle Odds'] = pd.to_numeric(df['Pinnacle Odds'])
            df = df.sort_values(by = "Pinnacle Odds", ascending = True, ignore_index = True)
            all_data.append(df)
        except:
            continue
        

        
    final_df = pd.concat(all_data)
    final_df.to_csv("Pinnacle NBA.csv")
    
scraper()

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3965678560.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3965678560.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3965678560.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3965678560.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\3965678560.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh

In [54]:
pin = pd.read_csv("Pinnacle NBA.csv")
pin = pin.drop(columns = "Unnamed: 0")

In [55]:
points = pin[pin["Prop"] == "Points"]
points = points.drop(columns = ["Prop"])
points.to_csv("Pinnacle Points.csv")
assists = pin[pin["Prop"] == "Assists"]
assists = assists.drop(columns = ["Prop"])
assists.to_csv("Pinnacle Assists.csv")
rebounds = pin[pin["Prop"] == "Rebounds"]
rebounds = rebounds.drop(columns = ["Prop"])
rebounds.to_csv("Pinnacle Rebounds.csv")
pra = pin[pin["Prop"] == "Pts+Rebs+Asts"]
pra = pra.drop(columns = ["Prop"])
pra.to_csv("Pinnacle PRA.csv")
to = pin[pin["Prop"] == "Turnovers"]
to = to.drop(columns = ["Prop"])
to.to_csv("Pinnacle Turnovers.csv")
tpg = pin[pin["Prop"] == "3pm"]
tpg = tpg.drop(columns = ["Prop"])
tpg.to_csv("Pinnacle 3 Point FG.csv")
stlblk = pin[pin["Prop"] == "Steals+Blocks"]
stlblk = stlblk.drop(columns = ["Prop"])
stlblk.to_csv("Pinnacle Steals+Blocks.csv")
blk = pin[pin["Prop"] == "Blocks"]
blk = blk.drop(columns = ["Prop"])
blk.to_csv("Pinnacle Blocks.csv")

In [56]:
df = pd.read_csv("player-points DK.csv")
df = df.drop(columns = "Unnamed: 0")
df1 = pd.read_csv("player-rebounds DK.csv")
df1 = df1.drop(columns = "Unnamed: 0")
df2 = pd.read_csv("player-assists DK.csv")
df2 = df2.drop(columns = "Unnamed: 0")
df3 = pd.read_csv("player-threes DK.csv")
df3 = df3.drop(columns = "Unnamed: 0")
df4 = pd.read_csv("player-turnovers DK.csv")
df4 = df4.drop(columns = "Unnamed: 0")
df5 = pd.read_csv("pts-%2B-reb-%2B-ast DK.csv")
df5 = df5.drop(columns = "Unnamed: 0")
df6 = pd.read_csv("pts-%2b-reb DK.csv")
df6 = df6.drop(columns = "Unnamed: 0")
df7 = pd.read_csv("pts-%2b-ast DK.csv")
df7 = df7.drop(columns = "Unnamed: 0")
df8 = pd.read_csv("ast-%2b-reb DK.csv")
df8 = df8.drop(columns = "Unnamed: 0")

In [57]:
pf = pd.read_csv("Points Prizepicks.csv")
pf = pf.drop(columns = "Unnamed: 0")
pf1 = pd.read_csv("Rebounds Prizepicks.csv")
pf1 = pf1.drop(columns = "Unnamed: 0")
pf2 = pd.read_csv("Assists Prizepicks.csv")
pf2 = pf2.drop(columns = "Unnamed: 0")
pf3 = pd.read_csv("3-PT Made Prizepicks.csv")
pf3 = pf3.drop(columns = "Unnamed: 0")
pf4 = pd.read_csv("Turnovers Prizepicks.csv")
pf4 = pf4.drop(columns = "Unnamed: 0")
pf5 = pd.read_csv("Pts+Rebs+Asts Prizepicks.csv")
pf5 = pf5.drop(columns = "Unnamed: 0")
pf6 = pd.read_csv("Pts+Rebs Prizepicks.csv")
pf6 = pf6.drop(columns = "Unnamed: 0")
pf7 = pd.read_csv("Pts+Asts Prizepicks.csv")
pf7 = pf7.drop(columns = "Unnamed: 0")
pf8 = pd.read_csv("Rebs+Asts Prizepicks.csv")
pf8 = pf8.drop(columns = "Unnamed: 0")

In [58]:
pf

,Name,Position,Team,Opp Team,Points
0,Luka Doncic,F-G,DAL,NOP,33.5
1,LeBron James\t,F,LAL,SAC,32.5
2,Jayson Tatum,F-G,BOS,SAS,30.5
3,Jordan Poole,G,GSW,ORL,27.5
4,Lauri Markkanen,F-C,UTA,CHI,27.5
5,Jaylen Brown,G-F,BOS,SAS,26.5
6,DeMar DeRozan,G-F,CHI,UTA,26.5
7,De'Aaron Fox,G,SAC,LAL,25.5
8,Klay Thompson,G,GSW,ORL,25.5
9,Zach LaVine,G-F,CHI,UTA,23.5


,Name,Position,Team,Opp Team,Points
0,Luka Doncic,F-G,DAL,NOP,33.5
1,LeBron James\t,F,LAL,SAC,32.5
2,Jayson Tatum,F-G,BOS,SAS,30.5
3,Jordan Poole,G,GSW,ORL,27.5
4,Lauri Markkanen,F-C,UTA,CHI,27.5
5,Jaylen Brown,G-F,BOS,SAS,26.5
6,DeMar DeRozan,G-F,CHI,UTA,26.5
7,De'Aaron Fox,G,SAC,LAL,25.5
8,Klay Thompson,G,GSW,ORL,25.5
9,Zach LaVine,G-F,CHI,UTA,23.5


In [59]:
pin1 = pd.read_csv("Pinnacle Points.csv")
pin1 = pin1.drop(columns = "Unnamed: 0")
pin1 = pin1.set_index("Name")
pin2 = pd.read_csv("Pinnacle Rebounds.csv")
pin2 = pin2.drop(columns = "Unnamed: 0")
pin2 = pin2.set_index("Name")
pin3 = pd.read_csv("Pinnacle Assists.csv")
pin3 = pin3.drop(columns = "Unnamed: 0")
pin3 = pin3.set_index("Name")
pin4 = pd.read_csv("Pinnacle 3pm.csv")
pin4 = pin4.drop(columns = "Unnamed: 0")
pin4 = pin4.set_index("Name")
pin5 = pd.read_csv("Pinnacle Turnovers.csv")
pin5 = pin5.drop(columns = "Unnamed: 0")
pin5 = pin5.set_index("Name")
pin6 = pd.read_csv("Pinnacle Pts+Rebs+Asts.csv")
pin6 = pin6.drop(columns = "Unnamed: 0")
pin6 = pin6.set_index("Name")

In [60]:
a = pd.merge(pf, df)
a = a.set_index("Name")
a1 = pd.merge(pf1,df1)
a1 = a1.set_index("Name")
a2 = pd.merge(pf2, df2)
a2 = a2.set_index("Name")
a3 = pd.merge(pf3,df3)
a3 = a3.set_index("Name")
a4 = pd.merge(pf4, df4)
a4 = a4.set_index("Name")
a5 = pd.merge(pf5,df5)
a5 = a5.set_index("Name")
a6 = pd.merge(pf6, df6)
a6 = a6.set_index("Name")
a7 = pd.merge(pf7,df7)
a7 = a7.set_index("Name")
a8 = pd.merge(pf8,df8)
a8 = a8.set_index("Name")

In [61]:
b = pd.merge(left=a, right=pin1, left_index=True, right_index=True, how="outer")
b = b.dropna(subset=["Points"])
b.fillna("", inplace=True)
b1 = pd.merge(left=a1, right=pin2, left_index=True, right_index=True, how="outer")
b1 = b1.dropna(subset=["Rebounds"])
b1.fillna("", inplace=True)
b2 = pd.merge(left=a2, right=pin3, left_index=True, right_index=True, how="outer")
b2 = b2.dropna(subset=["Assists"])
b2.fillna("", inplace=True)
b3 = pd.merge(left=a3, right=pin4, left_index=True, right_index=True, how="outer")
b3 = b3.dropna(subset=["3-PT Made"])
b3.fillna("", inplace=True)
b4 = pd.merge(left=a4, right=pin5, left_index=True, right_index=True, how="outer")
b4 = b4.dropna(subset=["Turnovers"])
b4.fillna("", inplace=True)
b5 = pd.merge(left=a5, right=pin6, left_index=True, right_index=True, how="outer")
b5 = b5.dropna(subset=["Pts+Rebs+Asts"])
b5.fillna("", inplace=True)

In [62]:
b = b.reindex(columns = ["Position","Team","Opp Team","Points","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b = b.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b["Pin Line"] = pd.to_numeric(b["Pin Line"], errors="coerce")
b["Pin Diff"] = b["Pin Line"] - b["Points"]
b["DK Diff"] = b["DK Line"] - b["Points"]
b["Pinnacle Odds"] = pd.to_numeric(b["Pinnacle Odds"], errors="coerce")
b["Odds Average"] = (b["DK Odds"] + b["Pinnacle Odds"])/2
b["Odds Average"].fillna(b["DK Odds"], inplace=True)
b["Edge"] = round(b["DK Odds"].abs()/(b["DK Odds"].abs() + 100),2)
b.fillna("", inplace=True)
b

,Position,Team,Opp Team,Points,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jeremy Sochan,F,SAS,BOS,11.5,11.5,11.5,-150.0,-128.0,0.0,0.0,-139.0,0.60
Kevon Looney,C-F,GSW,ORL,7.5,7.5,7.5,-135.0,-142.0,0.0,0.0,-138.5,0.57
Tim Hardaway Jr.,G-F,DAL,NOP,15.5,15.5,15.5,-135.0,-141.0,0.0,0.0,-138.0,0.57
Draymond Green,F,GSW,ORL,8.0,7.5,7.5,-135.0,-140.0,-0.5,-0.5,-137.5,0.57
Malik Beasley,G,UTA,CHI,15.5,15.5,,-135.0,,,0.0,-135.0,0.57
Nikola Vucevic,C,CHI,UTA,17.5,17.5,17.5,-130.0,-131.0,0.0,0.0,-130.5,0.57
Mike Conley,G,UTA,CHI,10.5,10.5,10.5,-130.0,-130.0,0.0,0.0,-130.0,0.57
Naji Marshall,F,NOP,DAL,15.5,15.5,15.5,-130.0,-130.0,0.0,0.0,-130.0,0.57
Markelle Fultz,G,ORL,GSW,11.5,11.5,11.5,-130.0,-119.0,0.0,0.0,-124.5,0.57


,Position,Team,Opp Team,Points,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jeremy Sochan,F,SAS,BOS,11.5,11.5,11.5,-150.0,-128.0,0.0,0.0,-139.0,0.60
Kevon Looney,C-F,GSW,ORL,7.5,7.5,7.5,-135.0,-142.0,0.0,0.0,-138.5,0.57
Tim Hardaway Jr.,G-F,DAL,NOP,15.5,15.5,15.5,-135.0,-141.0,0.0,0.0,-138.0,0.57
Draymond Green,F,GSW,ORL,8.0,7.5,7.5,-135.0,-140.0,-0.5,-0.5,-137.5,0.57
Malik Beasley,G,UTA,CHI,15.5,15.5,,-135.0,,,0.0,-135.0,0.57
Nikola Vucevic,C,CHI,UTA,17.5,17.5,17.5,-130.0,-131.0,0.0,0.0,-130.5,0.57
Mike Conley,G,UTA,CHI,10.5,10.5,10.5,-130.0,-130.0,0.0,0.0,-130.0,0.57
Naji Marshall,F,NOP,DAL,15.5,15.5,15.5,-130.0,-130.0,0.0,0.0,-130.0,0.57
Markelle Fultz,G,ORL,GSW,11.5,11.5,11.5,-130.0,-119.0,0.0,0.0,-124.5,0.57


In [63]:
b1 = b1.reindex(columns = ["Position","Team","Opp Team","Rebounds","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b1 = b1.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b1["Pin Line"] = pd.to_numeric(b1["Pin Line"], errors="coerce")
b1["Pin Diff"] = b1["Pin Line"] - b1["Rebounds"]
b1["DK Diff"] = b1["DK Line"] - b1["Rebounds"]
b1["Pinnacle Odds"] = pd.to_numeric(b1["Pinnacle Odds"], errors="coerce")
b1["Odds Average"] = (b1["DK Odds"] + b1["Pinnacle Odds"])/2
b1["Odds Average"].fillna(b1["DK Odds"], inplace=True)
b1["Edge"] = round(b1["DK Odds"].abs()/(b1["DK Odds"].abs() + 100),2)
b1.fillna("", inplace=True)
b1

,Position,Team,Opp Team,Rebounds,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Patrick Williams,F,CHI,UTA,5.0,4.5,4.5,-145.0,-153,-0.5,-0.5,-149.0,0.59
Al Horford,C-F,BOS,SAS,6.0,5.5,5.5,-140.0,-148,-0.5,-0.5,-144.0,0.58
Draymond Green,F,GSW,ORL,7.5,7.5,7.5,-135.0,-141,0.0,0.0,-138.0,0.57
Domantas Sabonis,F-C,SAC,LAL,13.5,13.5,13.5,-135.0,-136,0.0,0.0,-135.5,0.57
Luka Doncic,F-G,DAL,NOP,8.5,8.5,8.5,-135.0,-136,0.0,0.0,-135.5,0.57
Jayson Tatum,F-G,BOS,SAS,7.5,7.5,7.5,-130.0,-126,0.0,0.0,-128.0,0.57
Kevon Looney,C-F,GSW,ORL,10.5,10.5,10.5,-130.0,-106,0.0,0.0,-118.0,0.57
Paolo Banchero,F,ORL,GSW,6.5,6.5,6.5,-125.0,-131,0.0,0.0,-128.0,0.56
Tim Hardaway Jr.,G-F,DAL,NOP,3.5,3.5,3.5,-125.0,-130,0.0,0.0,-127.5,0.56


,Position,Team,Opp Team,Rebounds,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Patrick Williams,F,CHI,UTA,5.0,4.5,4.5,-145.0,-153,-0.5,-0.5,-149.0,0.59
Al Horford,C-F,BOS,SAS,6.0,5.5,5.5,-140.0,-148,-0.5,-0.5,-144.0,0.58
Draymond Green,F,GSW,ORL,7.5,7.5,7.5,-135.0,-141,0.0,0.0,-138.0,0.57
Domantas Sabonis,F-C,SAC,LAL,13.5,13.5,13.5,-135.0,-136,0.0,0.0,-135.5,0.57
Luka Doncic,F-G,DAL,NOP,8.5,8.5,8.5,-135.0,-136,0.0,0.0,-135.5,0.57
Jayson Tatum,F-G,BOS,SAS,7.5,7.5,7.5,-130.0,-126,0.0,0.0,-128.0,0.57
Kevon Looney,C-F,GSW,ORL,10.5,10.5,10.5,-130.0,-106,0.0,0.0,-118.0,0.57
Paolo Banchero,F,ORL,GSW,6.5,6.5,6.5,-125.0,-131,0.0,0.0,-128.0,0.56
Tim Hardaway Jr.,G-F,DAL,NOP,3.5,3.5,3.5,-125.0,-130,0.0,0.0,-127.5,0.56


In [64]:
b2 = b2.reindex(columns = ["Position","Team","Opp Team","Assists","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b2 = b2.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b2["Pin Line"] = pd.to_numeric(b2["Pin Line"], errors="coerce")
b2["Pin Diff"] = b2["Pin Line"] - b2["Assists"]
b2["DK Diff"] = b2["DK Line"] - b2["Assists"]
b2["Pinnacle Odds"] = pd.to_numeric(b2["Pinnacle Odds"], errors="coerce")
b2["Odds Average"] = (b2["DK Odds"] + b2["Pinnacle Odds"])/2
b2["Odds Average"].fillna(b2["DK Odds"], inplace=True)
b2["Edge"] = round(b2["DK Odds"].abs()/(b2["DK Odds"].abs() + 100),2)
b2.fillna("", inplace=True)
b2

,Position,Team,Opp Team,Assists,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Marcus Smart,G,BOS,SAS,7.0,6.5,6.5,-150.0,-159,-0.5,-0.5,-154.5,0.60
Spencer Dinwiddie,G,DAL,NOP,5.0,4.5,4.5,-140.0,-161,-0.5,-0.5,-150.5,0.58
Markelle Fultz,G,ORL,GSW,5.0,4.5,4.5,-140.0,-144,-0.5,-0.5,-142.0,0.58
Kevon Looney,C-F,GSW,ORL,2.5,2.5,2.5,-130.0,-136,0.0,0.0,-133.0,0.57
Klay Thompson,G,GSW,ORL,2.5,2.5,2.5,-130.0,-134,0.0,0.0,-132.0,0.57
DeMar DeRozan,G-F,CHI,UTA,4.5,4.5,4.5,-125.0,-130,0.0,0.0,-127.5,0.56
Paolo Banchero,F,ORL,GSW,3.5,3.5,3.5,-125.0,-129,0.0,0.0,-127.0,0.56
Trey Murphy III,F,NOP,DAL,1.5,1.5,1.5,-125.0,-127,0.0,0.0,-126.0,0.56
Draymond Green,F,GSW,ORL,6.5,6.5,6.5,-125.0,-121,0.0,0.0,-123.0,0.56


,Position,Team,Opp Team,Assists,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Marcus Smart,G,BOS,SAS,7.0,6.5,6.5,-150.0,-159,-0.5,-0.5,-154.5,0.60
Spencer Dinwiddie,G,DAL,NOP,5.0,4.5,4.5,-140.0,-161,-0.5,-0.5,-150.5,0.58
Markelle Fultz,G,ORL,GSW,5.0,4.5,4.5,-140.0,-144,-0.5,-0.5,-142.0,0.58
Kevon Looney,C-F,GSW,ORL,2.5,2.5,2.5,-130.0,-136,0.0,0.0,-133.0,0.57
Klay Thompson,G,GSW,ORL,2.5,2.5,2.5,-130.0,-134,0.0,0.0,-132.0,0.57
DeMar DeRozan,G-F,CHI,UTA,4.5,4.5,4.5,-125.0,-130,0.0,0.0,-127.5,0.56
Paolo Banchero,F,ORL,GSW,3.5,3.5,3.5,-125.0,-129,0.0,0.0,-127.0,0.56
Trey Murphy III,F,NOP,DAL,1.5,1.5,1.5,-125.0,-127,0.0,0.0,-126.0,0.56
Draymond Green,F,GSW,ORL,6.5,6.5,6.5,-125.0,-121,0.0,0.0,-123.0,0.56


In [65]:
b3 = b3.reindex(columns = ["Position","Team","Opp Team","3-PT Made","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b3 = b3.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b3["Pin Line"] = pd.to_numeric(b3["Pin Line"], errors="coerce")
b3["Pin Diff"] = b3["Pin Line"] - b3["3-PT Made"]
b3["DK Diff"] = b3["DK Line"] - b3["3-PT Made"]
b3["Pinnacle Odds"] = pd.to_numeric(b3["Pinnacle Odds"], errors="coerce")
b3["Odds Average"] = (b3["DK Odds"] + b3["Pinnacle Odds"])/2
b3["Odds Average"].fillna(b3["DK Odds"], inplace=True)
b3["Edge"] = round(b3["DK Odds"].abs()/(b3["DK Odds"].abs() + 100),2)
b3.fillna("", inplace=True)
b3

,Position,Team,Opp Team,3-PT Made,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
De'Aaron Fox,G,SAC,LAL,1.5,1.5,,-135.0,,,0.0,-135.0,0.57
Al Horford,C-F,BOS,SAS,1.5,1.5,1.5,-130.0,-213.0,0.0,0.0,-171.5,0.57
Zach LaVine,G-F,CHI,UTA,2.5,2.5,,-125.0,,,0.0,-125.0,0.56
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-152.0,0.0,0.0,-136.0,0.55
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-152.0,0.0,0.0,-136.0,0.55
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-119.0,0.0,0.0,-119.5,0.55
Franz Wagner,F,ORL,GSW,1.5,1.5,,-120.0,,,0.0,-120.0,0.55
Jose Alvarado,G,NOP,DAL,1.5,1.5,,-120.0,,,0.0,-120.0,0.55
Mike Conley,G,UTA,CHI,1.5,1.5,,-120.0,,,0.0,-120.0,0.55


,Position,Team,Opp Team,3-PT Made,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
De'Aaron Fox,G,SAC,LAL,1.5,1.5,,-135.0,,,0.0,-135.0,0.57
Al Horford,C-F,BOS,SAS,1.5,1.5,1.5,-130.0,-213.0,0.0,0.0,-171.5,0.57
Zach LaVine,G-F,CHI,UTA,2.5,2.5,,-125.0,,,0.0,-125.0,0.56
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-152.0,0.0,0.0,-136.0,0.55
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-152.0,0.0,0.0,-136.0,0.55
Spencer Dinwiddie,G,DAL,NOP,2.5,2.5,2.5,-120.0,-119.0,0.0,0.0,-119.5,0.55
Franz Wagner,F,ORL,GSW,1.5,1.5,,-120.0,,,0.0,-120.0,0.55
Jose Alvarado,G,NOP,DAL,1.5,1.5,,-120.0,,,0.0,-120.0,0.55
Mike Conley,G,UTA,CHI,1.5,1.5,,-120.0,,,0.0,-120.0,0.55


In [66]:
b4 = b4.reindex(columns = ["Position","Team","Opp Team","Turnovers","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b4 = b4.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b4["Pin Line"] = pd.to_numeric(b4["Pin Line"], errors="coerce")
b4["Pin Diff"] = b4["Pin Line"] - b4["Turnovers"]
b4["DK Diff"] = b4["DK Line"] - b4["Turnovers"]
b4["Pinnacle Odds"] = pd.to_numeric(b4["Pinnacle Odds"], errors="coerce")
b4["Odds Average"] = (b4["DK Odds"] + b4["Pinnacle Odds"])/2
b4["Odds Average"].fillna(b4["DK Odds"], inplace=True)
b4["Edge"] = round(b4["DK Odds"].abs()/(b4["DK Odds"].abs() + 100),2)
b4.fillna("", inplace=True)
b4

,Position,Team,Opp Team,Turnovers,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jordan Poole,G,GSW,ORL,3.5,3.5,3.5,-135.0,-157,0.0,0.0,-146.0,0.57
Lauri Markkanen,F-C,UTA,CHI,1.5,1.5,1.5,-135.0,-139,0.0,0.0,-137.0,0.57
Domantas Sabonis,F-C,SAC,LAL,2.5,2.5,2.5,-135.0,-112,0.0,0.0,-123.5,0.57
Nikola Vucevic,C,CHI,UTA,1.5,1.5,1.5,-130.0,-130,0.0,0.0,-130.0,0.57
Paolo Banchero,F,ORL,GSW,2.5,2.5,2.5,-130.0,-127,0.0,0.0,-128.5,0.57
Christian Wood,F,DAL,NOP,1.5,1.5,1.5,-125.0,-127,0.0,0.0,-126.0,0.56
Jayson Tatum,F-G,BOS,SAS,2.5,2.5,2.5,-120.0,-122,0.0,0.0,-121.0,0.55
Kevin Huerter,G-F,SAC,LAL,1.5,1.5,1.5,-105.0,-106,0.0,0.0,-105.5,0.51
Al Horford,C-F,BOS,SAS,0.5,0.5,0.5,-105.0,-105,0.0,0.0,-105.0,0.51


,Position,Team,Opp Team,Turnovers,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jordan Poole,G,GSW,ORL,3.5,3.5,3.5,-135.0,-157,0.0,0.0,-146.0,0.57
Lauri Markkanen,F-C,UTA,CHI,1.5,1.5,1.5,-135.0,-139,0.0,0.0,-137.0,0.57
Domantas Sabonis,F-C,SAC,LAL,2.5,2.5,2.5,-135.0,-112,0.0,0.0,-123.5,0.57
Nikola Vucevic,C,CHI,UTA,1.5,1.5,1.5,-130.0,-130,0.0,0.0,-130.0,0.57
Paolo Banchero,F,ORL,GSW,2.5,2.5,2.5,-130.0,-127,0.0,0.0,-128.5,0.57
Christian Wood,F,DAL,NOP,1.5,1.5,1.5,-125.0,-127,0.0,0.0,-126.0,0.56
Jayson Tatum,F-G,BOS,SAS,2.5,2.5,2.5,-120.0,-122,0.0,0.0,-121.0,0.55
Kevin Huerter,G-F,SAC,LAL,1.5,1.5,1.5,-105.0,-106,0.0,0.0,-105.5,0.51
Al Horford,C-F,BOS,SAS,0.5,0.5,0.5,-105.0,-105,0.0,0.0,-105.0,0.51


In [67]:
b5 = b5.reindex(columns = ["Position","Team","Opp Team","Pts+Rebs+Asts","DK Line", "Pin Line", "DK Odds", "Pinnacle Odds"])
b5 = b5.sort_values(by = ["DK Odds", "Pinnacle Odds"], ascending = [True,True])
b5["Pin Line"] = pd.to_numeric(b5["Pin Line"], errors="coerce")
b5["Pin Diff"] = b5["Pin Line"] - b5["Pts+Rebs+Asts"]
b5["DK Diff"] = b5["DK Line"] - b5["Pts+Rebs+Asts"]
b5["Pinnacle Odds"] = pd.to_numeric(b5["Pinnacle Odds"], errors="coerce")
b5["Odds Average"] = (b5["DK Odds"] + b5["Pinnacle Odds"])/2
b5["Odds Average"].fillna(b5["DK Odds"], inplace=True)
b5["Edge"] = round(b5["DK Odds"].abs()/(b5["DK Odds"].abs() + 100),2)
b5.fillna("", inplace=True)
b5

,Position,Team,Opp Team,Pts+Rebs+Asts,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jeremy Sochan,F,SAS,BOS,19.5,20.5,,-145.0,-122.0,,1.0,-133.5,0.59
Jayson Tatum,F-G,BOS,SAS,42.5,41.5,,-130.0,,,-1.0,-130.0,0.57
Herbert Jones,F,NOP,DAL,18.5,18.5,,-125.0,,,0.0,-125.0,0.56
Jaylen Brown,G-F,BOS,SAS,35.5,35.5,,-125.0,,,0.0,-125.0,0.56
Kevin Huerter,G-F,SAC,LAL,22.5,22.5,,-125.0,,,0.0,-125.0,0.56
Mike Conley,G,UTA,CHI,20.5,20.5,,-125.0,,,0.0,-125.0,0.56
Zach LaVine,G-F,CHI,UTA,31.5,31.5,,-125.0,,,0.0,-125.0,0.56
DeMar DeRozan,G-F,CHI,UTA,36.5,36.5,,-120.0,,,0.0,-120.0,0.55
Domantas Sabonis,F-C,SAC,LAL,42.5,42.5,,-120.0,,,0.0,-120.0,0.55


,Position,Team,Opp Team,Pts+Rebs+Asts,DK Line,Pin Line,DK Odds,Pinnacle Odds,Pin Diff,DK Diff,Odds Average,Edge
Name,,,,,,,,,,,,
Jeremy Sochan,F,SAS,BOS,19.5,20.5,,-145.0,-122.0,,1.0,-133.5,0.59
Jayson Tatum,F-G,BOS,SAS,42.5,41.5,,-130.0,,,-1.0,-130.0,0.57
Herbert Jones,F,NOP,DAL,18.5,18.5,,-125.0,,,0.0,-125.0,0.56
Jaylen Brown,G-F,BOS,SAS,35.5,35.5,,-125.0,,,0.0,-125.0,0.56
Kevin Huerter,G-F,SAC,LAL,22.5,22.5,,-125.0,,,0.0,-125.0,0.56
Mike Conley,G,UTA,CHI,20.5,20.5,,-125.0,,,0.0,-125.0,0.56
Zach LaVine,G-F,CHI,UTA,31.5,31.5,,-125.0,,,0.0,-125.0,0.56
DeMar DeRozan,G-F,CHI,UTA,36.5,36.5,,-120.0,,,0.0,-120.0,0.55
Domantas Sabonis,F-C,SAC,LAL,42.5,42.5,,-120.0,,,0.0,-120.0,0.55


In [68]:
a6["Diff"] = a6["DK Line"] - a6["Pts+Rebs"]
a6 = a6.sort_values(by = ["DK Odds", "Diff"], ascending = [True,False])
a6["Edge"] = round(a6["DK Odds"].abs()/(a6["DK Odds"].abs() + 100),2)
a6

,Position,Team,Opp Team,Pts+Rebs,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,17.5,17.5,-145,0.0,0.59
Jayson Tatum,F-G,BOS,SAS,38.5,37.5,-130,-1.0,0.57
Kevin Huerter,G-F,SAC,LAL,19.5,19.5,-125,0.0,0.56
Trey Murphy III,F,NOP,DAL,19.5,19.5,-125,0.0,0.56
Malik Beasley,G,UTA,CHI,18.5,18.5,-125,0.0,0.56
Jose Alvarado,G,NOP,DAL,17.5,17.5,-125,0.0,0.56
Domantas Sabonis,F-C,SAC,LAL,35.5,35.5,-120,0.0,0.55
Jaylen Brown,G-F,BOS,SAS,32.5,32.5,-120,0.0,0.55
DeMar DeRozan,G-F,CHI,UTA,31.5,31.5,-120,0.0,0.55


,Position,Team,Opp Team,Pts+Rebs,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,17.5,17.5,-145,0.0,0.59
Jayson Tatum,F-G,BOS,SAS,38.5,37.5,-130,-1.0,0.57
Kevin Huerter,G-F,SAC,LAL,19.5,19.5,-125,0.0,0.56
Trey Murphy III,F,NOP,DAL,19.5,19.5,-125,0.0,0.56
Malik Beasley,G,UTA,CHI,18.5,18.5,-125,0.0,0.56
Jose Alvarado,G,NOP,DAL,17.5,17.5,-125,0.0,0.56
Domantas Sabonis,F-C,SAC,LAL,35.5,35.5,-120,0.0,0.55
Jaylen Brown,G-F,BOS,SAS,32.5,32.5,-120,0.0,0.55
DeMar DeRozan,G-F,CHI,UTA,31.5,31.5,-120,0.0,0.55


In [69]:
a7["Diff"] = a7["DK Line"] - a7["Pts+Asts"]
a7 = a7.sort_values(by = ["DK Odds", "Diff"], ascending = [True,False])
a7["Edge"] = round(a7["DK Odds"].abs()/(a7["DK Odds"].abs() + 100),2)
a7

,Position,Team,Opp Team,Pts+Asts,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,13.5,14.5,-150,1.0,0.60
Jayson Tatum,F-G,BOS,SAS,34.5,33.5,-145,-1.0,0.59
Jordan Clarkson,G,UTA,CHI,26.5,26.5,-125,0.0,0.56
Wendell Carter Jr.,C-F,ORL,GSW,16.5,16.5,-125,0.0,0.56
Markelle Fultz,G,ORL,GSW,16.5,16.5,-125,0.0,0.56
Trey Murphy III,F,NOP,DAL,16.5,16.5,-125,0.0,0.56
Kevon Looney,C-F,GSW,ORL,10.5,10.5,-125,0.0,0.56
Christian Wood,F,DAL,NOP,21.5,20.5,-125,-1.0,0.56
Luka Doncic,F-G,DAL,NOP,43.5,43.5,-120,0.0,0.55


,Position,Team,Opp Team,Pts+Asts,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,13.5,14.5,-150,1.0,0.60
Jayson Tatum,F-G,BOS,SAS,34.5,33.5,-145,-1.0,0.59
Jordan Clarkson,G,UTA,CHI,26.5,26.5,-125,0.0,0.56
Wendell Carter Jr.,C-F,ORL,GSW,16.5,16.5,-125,0.0,0.56
Markelle Fultz,G,ORL,GSW,16.5,16.5,-125,0.0,0.56
Trey Murphy III,F,NOP,DAL,16.5,16.5,-125,0.0,0.56
Kevon Looney,C-F,GSW,ORL,10.5,10.5,-125,0.0,0.56
Christian Wood,F,DAL,NOP,21.5,20.5,-125,-1.0,0.56
Luka Doncic,F-G,DAL,NOP,43.5,43.5,-120,0.0,0.55


In [70]:
a8["Diff"] = a8["DK Line"] - a8["Rebs+Asts"]
a8 = a8.sort_values(by = ["DK Odds", "Diff"], ascending = [True,False])
a8["Edge"] = round(a8["DK Odds"].abs()/(a8["DK Odds"].abs() + 100),2)
a8

,Position,Team,Opp Team,Rebs+Asts,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,8.5,8.5,-140,0.0,0.58
Luka Doncic,F-G,DAL,NOP,17.5,17.5,-135,0.0,0.57
Nikola Vucevic,C,CHI,UTA,14.5,14.5,-130,0.0,0.57
Mike Conley,G,UTA,CHI,9.5,9.5,-130,0.0,0.57
Jordan Poole,G,GSW,ORL,8.5,8.5,-125,0.0,0.56
Zach LaVine,G-F,CHI,UTA,8.5,8.5,-125,0.0,0.56
Draymond Green,F,GSW,ORL,14.5,14.5,-120,0.0,0.55
Kevon Looney,C-F,GSW,ORL,13.5,13.5,-120,0.0,0.55
Paolo Banchero,F,ORL,GSW,10.5,10.5,-120,0.0,0.55


,Position,Team,Opp Team,Rebs+Asts,DK Line,DK Odds,Diff,Edge
Name,,,,,,,,
Jeremy Sochan,F,SAS,BOS,8.5,8.5,-140,0.0,0.58
Luka Doncic,F-G,DAL,NOP,17.5,17.5,-135,0.0,0.57
Nikola Vucevic,C,CHI,UTA,14.5,14.5,-130,0.0,0.57
Mike Conley,G,UTA,CHI,9.5,9.5,-130,0.0,0.57
Jordan Poole,G,GSW,ORL,8.5,8.5,-125,0.0,0.56
Zach LaVine,G-F,CHI,UTA,8.5,8.5,-125,0.0,0.56
Draymond Green,F,GSW,ORL,14.5,14.5,-120,0.0,0.55
Kevon Looney,C-F,GSW,ORL,13.5,13.5,-120,0.0,0.55
Paolo Banchero,F,ORL,GSW,10.5,10.5,-120,0.0,0.55


In [71]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://hashtagbasketball.com/nba-defense-vs-position")
positions = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[1]')))
teams = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[2]')))
pts = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[3]')))
fgper = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[4]')))
ftper = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[5]')))
threepm = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[6]')))
reb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[7]')))
ast = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[8]')))
stl = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[9]')))
blk = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[10]')))
to = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="ContentPlaceHolder1_GridView1"]/tbody/tr/td[11]')))

final_df = []
for i in range(len(teams)):
    data = {"Position":positions[i].text,
           "Team":teams[i].text,
           "Points":pts[i].text,
           "FG":fgper[i].text,
           "FT":ftper[i].text,
           "TPM":threepm[i].text,
           "Rebounds":reb[i].text,
           "Assists":ast[i].text,
           "Steals":stl[i].text,
           "Blocks":blk[i].text,
           "Turnovers":to[i].text}
    final_df.append(data)
driver.quit()

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\731529781.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\731529781.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [72]:
defdata = pd.DataFrame(final_df)
defdata[['Team','Team Def Rank']] = defdata.Team.str.split(expand=True)
defdata[['Points','PPG Rank']] = defdata.Points.str.split(expand=True)
defdata[['FG','FG% Rank']] = defdata.FG.str.split(expand=True)
defdata[['FT','FT% Rank']] = defdata.FT.str.split(expand=True)
defdata[['3PM','3PM Rank']] = defdata.TPM.str.split(expand=True)
defdata[['Rebounds','Rebounds Rank']] = defdata.Rebounds.str.split(expand=True)
defdata[['Assists','Assists Rank']] = defdata.Assists.str.split(expand=True)
defdata[['Steals','Steals Rank']] = defdata.Steals.str.split(expand=True)
defdata[['Blocks','Blocks Rank']] = defdata.Blocks.str.split(expand=True)
defdata[['Turnovers','Turnovers Rank']] = defdata.Turnovers.str.split(expand=True)
defdata = defdata.drop(columns = "TPM")

In [73]:
defdata.columns

Index(['Position', 'Team', 'Points', 'FG', 'FT', 'Rebounds', 'Assists',
       'Steals', 'Blocks', 'Turnovers', 'Team Def Rank', 'PPG Rank',
       'FG% Rank', 'FT% Rank', '3PM', '3PM Rank', 'Rebounds Rank',
       'Assists Rank', 'Steals Rank', 'Blocks Rank', 'Turnovers Rank'],
      dtype='object')

Index(['Position', 'Team', 'Points', 'FG', 'FT', 'Rebounds', 'Assists',
       'Steals', 'Blocks', 'Turnovers', 'Team Def Rank', 'PPG Rank',
       'FG% Rank', 'FT% Rank', '3PM', '3PM Rank', 'Rebounds Rank',
       'Assists Rank', 'Steals Rank', 'Blocks Rank', 'Turnovers Rank'],
      dtype='object')

In [74]:
defdata = defdata[["Position","Team", "Team Def Rank", "Points", "PPG Rank", "FG", "FG% Rank", "FT", "FT% Rank",
                  "3PM","3PM Rank", "Rebounds", "Rebounds Rank", "Assists","Assists Rank", "Steals","Steals Rank", "Blocks",
                  "Blocks Rank","Turnovers","Turnovers Rank"]]

In [75]:
cols = ["Team Def Rank", "Points", "PPG Rank", "FG", "FG% Rank", "FT", "FT% Rank", "3PM", "3PM Rank", "Rebounds", "Rebounds Rank",
       "Assists","Assists Rank","Steals","Steals Rank","Blocks","Blocks Rank","Turnovers","Turnovers Rank"]

In [76]:
defdata[cols] = defdata[cols].apply(pd.to_numeric, errors='coerce', axis=1)
defdata.to_csv("defdata.csv")

In [77]:
PG = defdata[defdata["Position"] == "PG"]

In [78]:
colormap = sns.color_palette("flare", as_cmap=True)
PG = PG.sort_values(by = "Team Def Rank", ascending = False)
PG = PG.style.background_gradient(cmap = colormap, subset = cols)
PG

,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
149,PG,DET,150.000000,25.300000,144.000000,46.200000,67.000000,82.700000,126.000000,2.700000,102.000000,6.200000,30.000000,9.100000,147.000000,2.000000,148.000000,0.600000,52.000000,2.900000,99.000000
138,PG,HOU,139.000000,24.000000,116.000000,44.800000,43.000000,81.400000,112.000000,3.400000,144.000000,4.700000,1.000000,8.500000,142.000000,2.500000,150.000000,0.600000,54.000000,2.500000,44.000000
131,PG,DEN,132.000000,27.300000,150.000000,48.800000,94.000000,85.600000,144.000000,3.100000,130.000000,5.500000,11.000000,9.500000,150.000000,1.200000,47.000000,0.400000,12.000000,2.800000,55.000000
126,PG,CHA,127.000000,25.300000,142.000000,46.200000,65.000000,83.200000,131.000000,2.800000,114.000000,6.200000,26.000000,8.400000,139.000000,1.300000,60.000000,0.800000,75.000000,3.900000,26.000000
124,PG,LAL,125.000000,26.600000,149.000000,45.900000,59.000000,80.300000,104.000000,3.000000,126.000000,5.700000,15.000000,8.800000,145.000000,1.600000,120.000000,0.500000,26.000000,3.500000,29.000000
123,PG,GSW,124.000000,24.600000,130.000000,44.000000,30.000000,83.000000,128.000000,2.900000,116.000000,6.400000,34.000000,8.800000,146.000000,1.900000,137.000000,0.500000,21.000000,3.300000,27.000000
117,PG,ORL,118.000000,24.200000,122.000000,45.600000,50.000000,87.800000,149.000000,2.800000,107.000000,5.500000,9.000000,9.100000,149.000000,1.800000,136.000000,0.400000,11.000000,3.500000,22.000000
115,PG,SAC,116.000000,24.500000,128.000000,45.300000,48.000000,88.200000,150.000000,2.800000,110.000000,5.400000,8.000000,7.800000,134.000000,2.000000,144.000000,0.300000,2.000000,3.600000,24.000000
107,PG,MIN,108.000000,25.400000,146.000000,43.300000,24.000000,79.400000,90.000000,3.100000,132.000000,6.600000,46.000000,7.700000,132.000000,1.700000,127.000000,0.400000,9.000000,3.700000,32.000000
105,PG,MEM,106.000000,22.100000,58.000000,41.600000,7.000000,83.300000,133.000000,2.800000,103.000000,7.000000,58.000000,7.400000,128.000000,1.600000,113.000000,0.600000,37.000000,2.900000,100.000000


,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
149,PG,DET,150.000000,25.300000,144.000000,46.200000,67.000000,82.700000,126.000000,2.700000,102.000000,6.200000,30.000000,9.100000,147.000000,2.000000,148.000000,0.600000,52.000000,2.900000,99.000000
138,PG,HOU,139.000000,24.000000,116.000000,44.800000,43.000000,81.400000,112.000000,3.400000,144.000000,4.700000,1.000000,8.500000,142.000000,2.500000,150.000000,0.600000,54.000000,2.500000,44.000000
131,PG,DEN,132.000000,27.300000,150.000000,48.800000,94.000000,85.600000,144.000000,3.100000,130.000000,5.500000,11.000000,9.500000,150.000000,1.200000,47.000000,0.400000,12.000000,2.800000,55.000000
126,PG,CHA,127.000000,25.300000,142.000000,46.200000,65.000000,83.200000,131.000000,2.800000,114.000000,6.200000,26.000000,8.400000,139.000000,1.300000,60.000000,0.800000,75.000000,3.900000,26.000000
124,PG,LAL,125.000000,26.600000,149.000000,45.900000,59.000000,80.300000,104.000000,3.000000,126.000000,5.700000,15.000000,8.800000,145.000000,1.600000,120.000000,0.500000,26.000000,3.500000,29.000000
123,PG,GSW,124.000000,24.600000,130.000000,44.000000,30.000000,83.000000,128.000000,2.900000,116.000000,6.400000,34.000000,8.800000,146.000000,1.900000,137.000000,0.500000,21.000000,3.300000,27.000000
117,PG,ORL,118.000000,24.200000,122.000000,45.600000,50.000000,87.800000,149.000000,2.800000,107.000000,5.500000,9.000000,9.100000,149.000000,1.800000,136.000000,0.400000,11.000000,3.500000,22.000000
115,PG,SAC,116.000000,24.500000,128.000000,45.300000,48.000000,88.200000,150.000000,2.800000,110.000000,5.400000,8.000000,7.800000,134.000000,2.000000,144.000000,0.300000,2.000000,3.600000,24.000000
107,PG,MIN,108.000000,25.400000,146.000000,43.300000,24.000000,79.400000,90.000000,3.100000,132.000000,6.600000,46.000000,7.700000,132.000000,1.700000,127.000000,0.400000,9.000000,3.700000,32.000000
105,PG,MEM,106.000000,22.100000,58.000000,41.600000,7.000000,83.300000,133.000000,2.800000,103.000000,7.000000,58.000000,7.400000,128.000000,1.600000,113.000000,0.600000,37.000000,2.900000,100.000000


In [79]:
SG = defdata[defdata["Position"] == "SG"]
SG = SG.sort_values(by = "Team Def Rank", ascending = False)
colormap = sns.color_palette("flare", as_cmap=True)
SG = SG.style.background_gradient(cmap = colormap, subset = cols)

In [80]:
SF = defdata[defdata["Position"] == "SF"]
SF = SF.sort_values(by = "Team Def Rank", ascending = False)
colormap = sns.color_palette("flare", as_cmap=True)
SF = SF.style.background_gradient(cmap = colormap, subset = cols)
SF

,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
127,SF,SAS,128.000000,21.600000,47.000000,52.400000,119.000000,83.800000,136.000000,2.600000,91.000000,8.400000,81.000000,4.200000,81.000000,1.500000,101.000000,0.600000,35.000000,3.000000,88.000000
118,SF,SAC,119.000000,22.300000,65.000000,47.300000,83.000000,81.500000,114.000000,2.400000,78.000000,8.100000,72.000000,4.200000,85.000000,1.500000,98.000000,0.900000,83.000000,2.500000,78.000000
97,SF,MEM,98.000000,22.300000,67.000000,44.400000,35.000000,80.200000,102.000000,2.800000,115.000000,7.600000,62.000000,4.600000,92.000000,1.500000,106.000000,1.000000,84.000000,2.500000,63.000000
96,SF,ORL,97.000000,22.700000,81.000000,48.100000,88.000000,79.300000,87.000000,3.000000,125.000000,8.300000,80.000000,3.900000,64.000000,1.400000,80.000000,0.800000,72.000000,2.100000,47.000000
82,SF,MIA,83.000000,20.500000,19.000000,47.500000,84.000000,79.000000,84.000000,2.600000,93.000000,8.200000,74.000000,4.300000,86.000000,1.500000,99.000000,0.700000,68.000000,2.600000,90.000000
80,SF,WAS,81.000000,21.700000,48.000000,45.900000,55.000000,82.500000,123.000000,2.500000,83.000000,8.200000,77.000000,3.800000,58.000000,1.400000,81.000000,0.900000,82.000000,2.000000,83.000000
77,SF,ATL,78.000000,23.600000,110.000000,48.200000,91.000000,77.400000,71.000000,2.600000,92.000000,9.500000,89.000000,3.600000,38.000000,1.100000,17.000000,1.100000,90.000000,2.900000,89.000000
73,SF,IND,74.000000,20.100000,17.000000,45.400000,49.000000,86.300000,146.000000,2.100000,62.000000,8.800000,84.000000,4.000000,66.000000,1.300000,50.000000,1.100000,89.000000,2.900000,119.000000
62,SF,NOP,63.000000,20.800000,24.000000,45.800000,54.000000,77.200000,68.000000,2.700000,95.000000,7.800000,67.000000,3.500000,35.000000,1.800000,134.000000,1.100000,94.000000,2.400000,92.000000
54,SF,OKL,55.000000,19.700000,14.000000,42.800000,18.000000,82.100000,118.000000,2.700000,96.000000,8.500000,83.000000,3.300000,19.000000,1.600000,117.000000,1.100000,95.000000,2.800000,93.000000


,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
127,SF,SAS,128.000000,21.600000,47.000000,52.400000,119.000000,83.800000,136.000000,2.600000,91.000000,8.400000,81.000000,4.200000,81.000000,1.500000,101.000000,0.600000,35.000000,3.000000,88.000000
118,SF,SAC,119.000000,22.300000,65.000000,47.300000,83.000000,81.500000,114.000000,2.400000,78.000000,8.100000,72.000000,4.200000,85.000000,1.500000,98.000000,0.900000,83.000000,2.500000,78.000000
97,SF,MEM,98.000000,22.300000,67.000000,44.400000,35.000000,80.200000,102.000000,2.800000,115.000000,7.600000,62.000000,4.600000,92.000000,1.500000,106.000000,1.000000,84.000000,2.500000,63.000000
96,SF,ORL,97.000000,22.700000,81.000000,48.100000,88.000000,79.300000,87.000000,3.000000,125.000000,8.300000,80.000000,3.900000,64.000000,1.400000,80.000000,0.800000,72.000000,2.100000,47.000000
82,SF,MIA,83.000000,20.500000,19.000000,47.500000,84.000000,79.000000,84.000000,2.600000,93.000000,8.200000,74.000000,4.300000,86.000000,1.500000,99.000000,0.700000,68.000000,2.600000,90.000000
80,SF,WAS,81.000000,21.700000,48.000000,45.900000,55.000000,82.500000,123.000000,2.500000,83.000000,8.200000,77.000000,3.800000,58.000000,1.400000,81.000000,0.900000,82.000000,2.000000,83.000000
77,SF,ATL,78.000000,23.600000,110.000000,48.200000,91.000000,77.400000,71.000000,2.600000,92.000000,9.500000,89.000000,3.600000,38.000000,1.100000,17.000000,1.100000,90.000000,2.900000,89.000000
73,SF,IND,74.000000,20.100000,17.000000,45.400000,49.000000,86.300000,146.000000,2.100000,62.000000,8.800000,84.000000,4.000000,66.000000,1.300000,50.000000,1.100000,89.000000,2.900000,119.000000
62,SF,NOP,63.000000,20.800000,24.000000,45.800000,54.000000,77.200000,68.000000,2.700000,95.000000,7.800000,67.000000,3.500000,35.000000,1.800000,134.000000,1.100000,94.000000,2.400000,92.000000
54,SF,OKL,55.000000,19.700000,14.000000,42.800000,18.000000,82.100000,118.000000,2.700000,96.000000,8.500000,83.000000,3.300000,19.000000,1.600000,117.000000,1.100000,95.000000,2.800000,93.000000


In [81]:
PF = defdata[defdata["Position"] == "PF"]
PF = PF.sort_values(by = "Team Def Rank", ascending = False)
colormap = sns.color_palette("flare", as_cmap=True)
PF = PF.style.background_gradient(cmap = colormap, subset = cols)
PF

,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
143,PF,LAC,144.000000,24.100000,119.000000,52.200000,118.000000,79.400000,88.000000,1.900000,49.000000,11.400000,106.000000,3.800000,56.000000,1.500000,85.000000,1.300000,103.000000,2.700000,107.000000
141,PF,ATL,142.000000,22.900000,88.000000,50.200000,104.000000,83.300000,134.000000,1.800000,41.000000,11.900000,113.000000,3.900000,62.000000,1.200000,40.000000,1.600000,117.000000,2.300000,122.000000
139,PF,MIN,140.000000,24.500000,127.000000,49.600000,99.000000,77.400000,72.000000,2.300000,68.000000,10.600000,95.000000,4.200000,80.000000,1.500000,100.000000,1.300000,101.000000,2.500000,69.000000
137,PF,SAS,138.000000,23.600000,111.000000,56.200000,134.000000,74.500000,40.000000,2.100000,60.000000,11.900000,115.000000,3.600000,44.000000,1.400000,82.000000,1.300000,102.000000,2.400000,117.000000
130,PF,DEN,131.000000,22.600000,77.000000,54.400000,127.000000,75.100000,44.000000,1.900000,52.000000,9.800000,91.000000,3.700000,50.000000,1.700000,123.000000,1.500000,115.000000,2.200000,109.000000
128,PF,DET,129.000000,24.400000,124.000000,56.000000,132.000000,76.000000,56.000000,2.000000,56.000000,11.900000,114.000000,3.600000,43.000000,1.100000,20.000000,1.800000,124.000000,2.600000,113.000000
119,PF,LAL,120.000000,22.200000,63.000000,49.800000,101.000000,71.800000,24.000000,1.900000,48.000000,10.900000,99.000000,4.100000,79.000000,1.800000,131.000000,1.400000,105.000000,2.100000,106.000000
111,PF,PHX,112.000000,24.300000,123.000000,55.800000,131.000000,75.500000,49.000000,1.600000,34.000000,11.200000,102.000000,3.800000,55.000000,1.900000,138.000000,1.300000,104.000000,3.100000,9.000000
110,PF,ORL,111.000000,22.000000,55.000000,49.800000,102.000000,79.700000,98.000000,1.700000,37.000000,11.800000,111.000000,3.400000,25.000000,1.400000,77.000000,1.500000,113.000000,2.400000,126.000000
101,PF,WAS,102.000000,23.300000,98.000000,49.100000,95.000000,82.700000,127.000000,2.000000,55.000000,11.300000,105.000000,3.000000,5.000000,1.100000,28.000000,1.400000,108.000000,2.000000,112.000000


,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
143,PF,LAC,144.000000,24.100000,119.000000,52.200000,118.000000,79.400000,88.000000,1.900000,49.000000,11.400000,106.000000,3.800000,56.000000,1.500000,85.000000,1.300000,103.000000,2.700000,107.000000
141,PF,ATL,142.000000,22.900000,88.000000,50.200000,104.000000,83.300000,134.000000,1.800000,41.000000,11.900000,113.000000,3.900000,62.000000,1.200000,40.000000,1.600000,117.000000,2.300000,122.000000
139,PF,MIN,140.000000,24.500000,127.000000,49.600000,99.000000,77.400000,72.000000,2.300000,68.000000,10.600000,95.000000,4.200000,80.000000,1.500000,100.000000,1.300000,101.000000,2.500000,69.000000
137,PF,SAS,138.000000,23.600000,111.000000,56.200000,134.000000,74.500000,40.000000,2.100000,60.000000,11.900000,115.000000,3.600000,44.000000,1.400000,82.000000,1.300000,102.000000,2.400000,117.000000
130,PF,DEN,131.000000,22.600000,77.000000,54.400000,127.000000,75.100000,44.000000,1.900000,52.000000,9.800000,91.000000,3.700000,50.000000,1.700000,123.000000,1.500000,115.000000,2.200000,109.000000
128,PF,DET,129.000000,24.400000,124.000000,56.000000,132.000000,76.000000,56.000000,2.000000,56.000000,11.900000,114.000000,3.600000,43.000000,1.100000,20.000000,1.800000,124.000000,2.600000,113.000000
119,PF,LAL,120.000000,22.200000,63.000000,49.800000,101.000000,71.800000,24.000000,1.900000,48.000000,10.900000,99.000000,4.100000,79.000000,1.800000,131.000000,1.400000,105.000000,2.100000,106.000000
111,PF,PHX,112.000000,24.300000,123.000000,55.800000,131.000000,75.500000,49.000000,1.600000,34.000000,11.200000,102.000000,3.800000,55.000000,1.900000,138.000000,1.300000,104.000000,3.100000,9.000000
110,PF,ORL,111.000000,22.000000,55.000000,49.800000,102.000000,79.700000,98.000000,1.700000,37.000000,11.800000,111.000000,3.400000,25.000000,1.400000,77.000000,1.500000,113.000000,2.400000,126.000000
101,PF,WAS,102.000000,23.300000,98.000000,49.100000,95.000000,82.700000,127.000000,2.000000,55.000000,11.300000,105.000000,3.000000,5.000000,1.100000,28.000000,1.400000,108.000000,2.000000,112.000000


In [82]:
C = defdata[defdata["Position"] == "C"]
C = C.sort_values(by = "Team Def Rank", ascending = False)
colormap = sns.color_palette("flare", as_cmap=True)
C = C.style.background_gradient(cmap = colormap, subset = cols)
C

,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
147,C,IND,148.000000,24.500000,126.000000,58.600000,142.000000,77.100000,67.000000,0.800000,5.000000,16.400000,147.000000,4.000000,72.000000,1.000000,12.000000,1.900000,136.000000,2.800000,147.000000
146,C,CHA,147.000000,25.600000,147.000000,51.500000,112.000000,76.300000,58.000000,1.500000,31.000000,16.900000,149.000000,3.700000,47.000000,1.000000,13.000000,2.600000,149.000000,2.800000,135.000000
145,C,POR,146.000000,24.900000,136.000000,57.800000,140.000000,69.000000,13.000000,1.100000,22.000000,15.200000,137.000000,4.100000,75.000000,1.100000,29.000000,2.300000,144.000000,2.800000,138.000000
140,C,LAL,141.000000,22.100000,57.000000,60.200000,146.000000,65.400000,5.000000,0.800000,8.000000,17.100000,150.000000,4.000000,71.000000,1.500000,87.000000,2.300000,146.000000,2.500000,150.000000
135,C,ATL,136.000000,24.600000,129.000000,57.300000,139.000000,73.300000,29.000000,1.400000,30.000000,15.500000,140.000000,3.300000,14.000000,1.300000,54.000000,1.900000,130.000000,2.600000,134.000000
134,C,NOP,135.000000,25.200000,141.000000,64.600000,150.000000,69.400000,15.000000,1.100000,21.000000,15.600000,141.000000,3.400000,27.000000,1.100000,22.000000,2.100000,143.000000,3.000000,137.000000
122,C,HOU,123.000000,22.100000,56.000000,55.500000,129.000000,78.400000,80.000000,1.000000,17.000000,13.800000,125.000000,3.700000,49.000000,1.000000,14.000000,3.100000,150.000000,2.900000,144.000000
121,C,MEM,122.000000,21.300000,37.000000,55.700000,130.000000,71.700000,23.000000,1.100000,20.000000,13.600000,124.000000,4.100000,73.000000,1.400000,79.000000,2.000000,139.000000,2.600000,136.000000
120,C,SAC,121.000000,22.600000,78.000000,59.800000,145.000000,73.900000,35.000000,0.800000,6.000000,15.100000,135.000000,4.200000,83.000000,0.900000,9.000000,1.700000,121.000000,3.000000,148.000000
114,C,WAS,115.000000,23.300000,101.000000,53.700000,125.000000,77.000000,66.000000,1.000000,13.000000,14.400000,131.000000,3.100000,13.000000,1.100000,27.000000,1.900000,132.000000,2.800000,140.000000


,Position,Team,Team Def Rank,Points,PPG Rank,FG,FG% Rank,FT,FT% Rank,3PM,3PM Rank,Rebounds,Rebounds Rank,Assists,Assists Rank,Steals,Steals Rank,Blocks,Blocks Rank,Turnovers,Turnovers Rank
147,C,IND,148.000000,24.500000,126.000000,58.600000,142.000000,77.100000,67.000000,0.800000,5.000000,16.400000,147.000000,4.000000,72.000000,1.000000,12.000000,1.900000,136.000000,2.800000,147.000000
146,C,CHA,147.000000,25.600000,147.000000,51.500000,112.000000,76.300000,58.000000,1.500000,31.000000,16.900000,149.000000,3.700000,47.000000,1.000000,13.000000,2.600000,149.000000,2.800000,135.000000
145,C,POR,146.000000,24.900000,136.000000,57.800000,140.000000,69.000000,13.000000,1.100000,22.000000,15.200000,137.000000,4.100000,75.000000,1.100000,29.000000,2.300000,144.000000,2.800000,138.000000
140,C,LAL,141.000000,22.100000,57.000000,60.200000,146.000000,65.400000,5.000000,0.800000,8.000000,17.100000,150.000000,4.000000,71.000000,1.500000,87.000000,2.300000,146.000000,2.500000,150.000000
135,C,ATL,136.000000,24.600000,129.000000,57.300000,139.000000,73.300000,29.000000,1.400000,30.000000,15.500000,140.000000,3.300000,14.000000,1.300000,54.000000,1.900000,130.000000,2.600000,134.000000
134,C,NOP,135.000000,25.200000,141.000000,64.600000,150.000000,69.400000,15.000000,1.100000,21.000000,15.600000,141.000000,3.400000,27.000000,1.100000,22.000000,2.100000,143.000000,3.000000,137.000000
122,C,HOU,123.000000,22.100000,56.000000,55.500000,129.000000,78.400000,80.000000,1.000000,17.000000,13.800000,125.000000,3.700000,49.000000,1.000000,14.000000,3.100000,150.000000,2.900000,144.000000
121,C,MEM,122.000000,21.300000,37.000000,55.700000,130.000000,71.700000,23.000000,1.100000,20.000000,13.600000,124.000000,4.100000,73.000000,1.400000,79.000000,2.000000,139.000000,2.600000,136.000000
120,C,SAC,121.000000,22.600000,78.000000,59.800000,145.000000,73.900000,35.000000,0.800000,6.000000,15.100000,135.000000,4.200000,83.000000,0.900000,9.000000,1.700000,121.000000,3.000000,148.000000
114,C,WAS,115.000000,23.300000,101.000000,53.700000,125.000000,77.000000,66.000000,1.000000,13.000000,14.400000,131.000000,3.100000,13.000000,1.100000,27.000000,1.900000,132.000000,2.800000,140.000000


In [83]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.nbastuffer.com/2022-2023-nba-player-stats/")
src = driver.page_source
soup = BeautifulSoup(src, "lxml")
table = soup.find("table", attrs = {"id": "tablepress-99"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers]
rows = table.findAll('tr')[1:]
playerstats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]
stats = pd.DataFrame(playerstats, columns = headerlist)
stats = stats.rename(columns = {"RANK":"Rank", "FULL NAME":"Name","TEAM":"Team","POS":"Pos","AGE":"Age","MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor":"Min%",
                           "TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions":"TO%",
                           "eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA":"eFG%",
                           "USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor":"USG%",
                           "TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.":"TS%",
                           "PPGPointsPoints per game.":"PPG","RPGReboundsRebounds per game.":"RPG", "TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.":"TRB%"
                           ,"APGAssistsAssists per game.":"APG", "AST%Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court":"AST%",
                           "SPGStealsSteals per game.":"SPG","BPGBlocksBlocks per game.":"BPG","TOPGTurnoversTurnovers per game.":"TOPG",
                           "VIVersatility IndexVersatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10":"VI",
                           "ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.":"ORTG",
                           "DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.":"DRTG"})

driver.quit()                                                             

C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\2726474931.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\Mani Suresh\AppData\Local\Temp\ipykernel_15920\2726474931.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [84]:
columns = ['GP', 'MPG', 'Min%',
       'USG%','TO%','FTA', 'FT%', '2PA', '2P%', '3PA',
       '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%',
       'SPG', 'BPG', 'TOPG',
       'VI', 'ORTG','DRTG']
stats[columns] = stats[columns].apply(pd.to_numeric, errors='coerce', axis=1)
stats = stats.sort_values(by = "VI", ascending = False)
stats.to_csv("stats.csv")

In [85]:
stats.columns

Index(['Rank', 'Name', 'Team', 'Pos', 'Age', 'GP', 'MPG', 'Min%', 'USG%',
       'TO%', 'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG',
       'RPG', 'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'TOPG', 'VI', 'ORTG',
       'DRTG'],
      dtype='object')

Index(['Rank', 'Name', 'Team', 'Pos', 'Age', 'GP', 'MPG', 'Min%', 'USG%',
       'TO%', 'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG',
       'RPG', 'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'TOPG', 'VI', 'ORTG',
       'DRTG'],
      dtype='object')

In [120]:
pointcombine = stats.iloc[:, [1,18, 6, 7, 8, 12, 13, 14, 15, 16,17,10,11,26,27]]
pointcombine = pointcombine.set_index("Name")
reboundcombine = stats.iloc[:, [1,19,20]]
reboundcombine = reboundcombine.set_index("Name")
assistcombine = stats.iloc[:,[1,21,22]]
assistcombine = assistcombine.set_index("Name")
tpmcombine = stats.iloc[:,[1,14,15]]
tpmcombine = tpmcombine.set_index("Name")
turnovercombine = stats.iloc[:,[1,25,9]]
turnovercombine = turnovercombine.set_index("Name")
pra = stats.iloc[:,[1, 18, 19,20,21,22, 8]]
pra = pra.set_index("Name")
pr = stats.iloc[:, [1,18,19,20, 8]]
pr = pr.set_index("Name")
pr = stats.iloc[:, [1, 18, 21, 22, 8]]
pr = pr.set_index("Name")
ra = stats.iloc[:, [1, 19,20,21,22,8]]
ra = ra.set_index("Name")
pa = stats.iloc[:, [1, 18, 19, 20, 8]]

In [121]:
f = pd.merge(left=b, right=pointcombine, left_index=True, right_index=True, how="inner")
f = f.reset_index()
f1 = pd.merge(left=b1, right=reboundcombine, left_index=True, right_index=True, how="inner")
f1 = f1.reset_index()
f2 = pd.merge(left=b2, right=assistcombine, left_index=True, right_index=True, how="inner")
f2 = f2.reset_index()
f3 = pd.merge(left=b3, right=tpmcombine, left_index=True, right_index=True, how="inner")
f3 = f3.reset_index()
f4 = pd.merge(left=b4, right=turnovercombine, left_index=True, right_index=True, how="inner")
f4 = f4.reset_index()
f5 = pd.merge(left=b5, right=pra, left_index=True, right_index=True, how="inner")
f5 = f5.reset_index()
f6 = pd.merge(left=a6, right=pr, left_index=True, right_index=True, how="inner")
f7 = pd.merge(left=a7, right=pa, left_index=True, right_index=True, how="inner")
f8 = pd.merge(left=a8, right=ra, left_index=True, right_index=True, how="inner")

In [124]:
f = f.sort_values(by = "Odds Average")
f1 = f1.sort_values(by = "Odds Average")
f2 = f2.sort_values(by = "Odds Average")
f3 = f3.sort_values(by = "Odds Average")
f4 = f4.sort_values(by = "Odds Average")
f5 = f5.sort_values(by = "Odds Average")
f6 = f6.sort_values(by = "DK Odds")
f7 = f7.sort_values(by = "DK Odds")
f8 = f8.sort_values(by = "DK Odds")

In [125]:
colormap = sns.color_palette("flare", as_cmap=True)
f['Pin Diff'] = pd.to_numeric(f['Pin Diff'], errors='coerce')
f = f.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])
f1['Pin Diff'] = pd.to_numeric(f1['Pin Diff'], errors='coerce')
f1 = f1.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])
f2['Pin Diff'] = pd.to_numeric(f2['Pin Diff'], errors='coerce')
f2 = f2.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])
f3['Pin Diff'] = pd.to_numeric(f3['Pin Diff'], errors='coerce')
f3 = f3.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])
f4['Pin Diff'] = pd.to_numeric(f4['Pin Diff'], errors='coerce')
f4 = f4.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])
f5['Pin Diff'] = pd.to_numeric(f5['Pin Diff'], errors='coerce')
f5 = f5.style.background_gradient(cmap=colormap, subset=["DK Diff","Pin Diff"])

In [126]:
# with pd.ExcelWriter("NBA_PROPS.xlsx", engine="openpyxl") as writer:
#     f.to_excel(writer, sheet_name="Points") 
#     f1.to_excel(writer, sheet_name="Rebounds")
#     f2.to_excel(writer, sheet_name="Assists")
#     f3.to_excel(writer, sheet_name="3-PT Made")
#     f4.to_excel(writer, sheet_name="Turnovers")
#     f5.to_excel(writer, sheet_name="Pts+Rbs+Asts")
#     f6.to_excel(writer, sheet_name="Pts+Rebs")
#     f7.to_excel(writer, sheet_name="Pts+Asts")
#     f8.to_excel(writer, sheet_name="Asts+Rebs")
#     stats.to_excel(writer, sheet_name="Player Stats")
#     PG.to_excel(writer, sheet_name="PG Defensive Stats")
#     SG.to_excel(writer, sheet_name="SG Defensive Stats")
#     SF.to_excel(writer, sheet_name="SF Defensive Stats")
#     PF.to_excel(writer, sheet_name="PF Defensive Stats")
#     C.to_excel(writer, sheet_name="C Defensive Stats")

with pd.ExcelWriter("NBA_PROPS.xlsx", engine="openpyxl") as writer:
    try:
        f.to_excel(writer, sheet_name="Points") 
    except ValueError:
        print("Sheet 'Points' is empty")

    try:
        f1.to_excel(writer, sheet_name="Rebounds")
    except ValueError:
        print("Sheet 'Rebounds' is empty")

    try:
        f2.to_excel(writer, sheet_name="Assists")
    except ValueError:
        print("Sheet 'Assists' is empty")

    try:
        f3.to_excel(writer, sheet_name="3-PT Made")
    except ValueError:
        print("Sheet '3-PT Made' is empty")

    try:
        f4.to_excel(writer, sheet_name="Turnovers")
    except ValueError:
        print("Sheet 'Turnovers' is empty")

    try:
        f5.to_excel(writer, sheet_name="Pts+Rbs+Asts")
    except ValueError:
        print("Sheet 'Pts+Rbs+Asts' is empty")

    try:
        f6.to_excel(writer, sheet_name="Pts+Rebs")
    except ValueError:
        print("Sheet 'Pts+Rebs' is empty")

    try:
        a7.to_excel(writer, sheet_name="Pts+Asts")
    except ValueError:
        print("Sheet 'Pts+Asts' is empty")

    try:
        f8.to_excel(writer, sheet_name="Asts+Rebs")
    except ValueError:
        print("Sheet 'Asts+Rebs' is empty")
    
    stats.to_excel(writer, sheet_name="Player Stats")
    PG.to_excel(writer, sheet_name="PG Defensive Stats")
    SG.to_excel(writer, sheet_name="SG Defensive Stats")
    SF.to_excel(writer, sheet_name="SF Defensive Stats")
    PF.to_excel(writer, sheet_name="PF Defensive Stats")
    C.to_excel(writer, sheet_name="C Defensive Stats")


C:\Users\Mani Suresh\anaconda3\lib\site-packages\pandas\io\formats\style.py:3554: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\Mani Suresh\anaconda3\lib\site-packages\pandas\io\formats\style.py:3555: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax
C:\Users\Mani Suresh\anaconda3\lib\site-packages\pandas\io\formats\style.py:3554: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\Mani Suresh\anaconda3\lib\site-packages\pandas\io\formats\style.py:3555: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


In [92]:
with pd.ExcelWriter("Draft Kings Odds.xlsx", engine="openpyxl") as writer:
    a.to_excel(writer, sheet_name="Points") 
    a1.to_excel(writer, sheet_name="Rebounds")
    a2.to_excel(writer, sheet_name="Assists")
    a3.to_excel(writer, sheet_name="3-PT Made")
    a4.to_excel(writer, sheet_name="Turnovers")
    a5.to_excel(writer, sheet_name="Pts+Rbs+Asts")
    a6.to_excel(writer, sheet_name="Pts+Rebs")
    a7.to_excel(writer, sheet_name="Pts+Asts")
    a8.to_excel(writer, sheet_name="Asts+Rebs")
    PG.to_excel(writer, sheet_name="PG Defensive Stats")
    SG.to_excel(writer, sheet_name="SG Defensive Stats")
    SF.to_excel(writer, sheet_name="SF Defensive Stats")
    PF.to_excel(writer, sheet_name="PF Defensive Stats")
    C.to_excel(writer, sheet_name="C Defensive Stats")
    